In [1]:
import os
import json
import psycopg2
from psycopg2.extras import DictCursor

In [2]:
POSTGRESQL_HOST = '10.129.0.25'

In [3]:
!pip install ipython-sql

In [4]:
%load_ext sql

In [5]:
CONNECT_DATA = 'postgresql://{}:{}@{}/{}'.format(
    os.environ['POSTGRESQL_USER'],
    os.environ['POSTGRESQL_PASSWORD'],
    POSTGRESQL_HOST,
    'demo'
)

In [6]:
%%sql $CONNECT_DATA

WITH number_of_tickets AS (
    SELECT ticket_no, COUNT(flight_id) AS flight_count
    FROM ticket_flights
    GROUP BY ticket_no
)
SELECT * FROM number_of_tickets
WHERE flight_count > 3
LIMIT 1;

1 rows affected.


ticket_no,flight_count
0005432312163,4


In [32]:
%%sql $CONNECT_DATA

WITH number_of_tickets AS (
    SELECT ticket_no, COUNT(flight_id) AS flight_count
    FROM ticket_flights
    GROUP BY ticket_no
)
SELECT COUNT(*) FROM number_of_tickets
WHERE flight_count = 5;

1 rows affected.


count
54


In [9]:
%%sql postgresql:///jovyan
CREATE TABLE Employees (
    EMPLOYEE_ID INT PRIMARY KEY,
    FIRST_NAME VARCHAR(100),
    LAST_NAME VARCHAR(100),
    HIRE_DATE DATE,
    JOB_ID VARCHAR(100),
    MANAGER_ID INT,
    DEPARTMENT_ID INT
);

CREATE TABLE Departments (
    DEPARTMENT_ID INT PRIMARY KEY,
    DEPARTMENT_NAME VARCHAR(100),
    MANAGER_ID INT
);
INSERT INTO Employees (EMPLOYEE_ID, FIRST_NAME, LAST_NAME, HIRE_DATE, JOB_ID, MANAGER_ID, DEPARTMENT_ID)
VALUES
(100, 'Steven', 'King', '1987-06-17', 'AD_PRES', 0, 90),
(101, 'Neena', 'Kochhar', '1987-06-18', 'AD_VP', 100, 90),
(102, 'Lex', 'DeHaan', '1987-06-19', 'AD_VP', 100, 90),
(103, 'Alexander', 'Hunold', '1987-06-20', 'IT_PROG', 102, 60),
(107, 'Diana', 'Lorentz', '1987-06-24', 'IT_PROG', 103, 60),
(108, 'Nancy', 'Greenberg', '1987-06-25', 'FI_MGR', 101, 100),
(109, 'Daniel', 'Faviet', '1987-06-26', 'FI_ACCOUNT', 108, 100),
(114, 'Den', 'Raphaely', '1987-07-01', 'PU_MAN', 100, 30),
(118, 'Guy', 'Himuro', '1987-07-05', 'PU_CLERK', 114, 30),
(144, 'Peter', 'Vargas', '1987-07-31', 'ST_CLERK', 114, 50),
(145, 'John', 'Russell', '1987-08-01', 'SA_MAN', 100, 80),
(146, 'Karen', 'Partners', '1987-08-02', 'SA_MAN', 100, 80);

INSERT INTO Departments (DEPARTMENT_ID, DEPARTMENT_NAME, MANAGER_ID)
VALUES
(10, 'Administration', 200),
(30, 'Purchasing', 114),
(40, 'HumanResources', 203),
(50, 'Shipping', 121),
(60, 'IT', 103),
(70, 'PublicRelations', 204),
(80, 'Sales', 145),
(90, 'Executive', 100),
(100, 'Finance', 108),
(110, 'Accounting', 205);

Done.
Done.
12 rows affected.
10 rows affected.


[]

In [10]:
%%sql postgresql:///jovyan
SELECT 
    EMPLOYEE_ID,
    FIRST_NAME,
    LAST_NAME,
    HIRE_DATE,
    JOB_ID,
    MANAGER_ID,
    DEPARTMENT_ID
FROM 
    Employees;

12 rows affected.


employee_id,first_name,last_name,hire_date,job_id,manager_id,department_id
100,Steven,King,1987-06-17,AD_PRES,0,90
101,Neena,Kochhar,1987-06-18,AD_VP,100,90
102,Lex,DeHaan,1987-06-19,AD_VP,100,90
103,Alexander,Hunold,1987-06-20,IT_PROG,102,60
107,Diana,Lorentz,1987-06-24,IT_PROG,103,60
108,Nancy,Greenberg,1987-06-25,FI_MGR,101,100
109,Daniel,Faviet,1987-06-26,FI_ACCOUNT,108,100
114,Den,Raphaely,1987-07-01,PU_MAN,100,30
118,Guy,Himuro,1987-07-05,PU_CLERK,114,30
144,Peter,Vargas,1987-07-31,ST_CLERK,114,50


In [11]:
%%sql postgresql:///jovyan
SELECT 
    DEPARTMENT_ID,
    DEPARTMENT_NAME,
    MANAGER_ID
FROM 
    Departments

10 rows affected.


department_id,department_name,manager_id
10,Administration,200
30,Purchasing,114
40,HumanResources,203
50,Shipping,121
60,IT,103
70,PublicRelations,204
80,Sales,145
90,Executive,100
100,Finance,108
110,Accounting,205


In [12]:
%%sql postgresql:///jovyan
SELECT 
    d.DEPARTMENT_NAME,
    e.FIRST_NAME,
    e.LAST_NAME
FROM 
    Employees e
JOIN 
    Departments d ON e.DEPARTMENT_ID = d.DEPARTMENT_ID
WHERE 
    e.HIRE_DATE = (SELECT MIN(HIRE_DATE) FROM Employees)

1 rows affected.


department_name,first_name,last_name
Executive,Steven,King


In [16]:
%%sql postgresql:///jovyan
 SELECT 
    d.DEPARTMENT_NAME, 
    CONCAT(e.FIRST_NAME, ' ', e.LAST_NAME) AS MANAGER_FULL_NAME,
    f.NUM_EMPLOYEES
FROM 
    Departments d
JOIN 
    Employees e 
ON 
    d.MANAGER_ID = e.EMPLOYEE_ID
JOIN
(SELECT DEPARTMENT_ID, COUNT(*) AS NUM_EMPLOYEES FROM Employees
GROUP BY 
    DEPARTMENT_ID) f
ON d. DEPARTMENT_ID=f. DEPARTMENT_ID ORDER BY d.DEPARTMENT_NAME

5 rows affected.


department_name,manager_full_name,num_employees
Executive,Steven King,3
Finance,Nancy Greenberg,2
IT,Alexander Hunold,2
Purchasing,Den Raphaely,2
Sales,John Russell,2
